In [ ]:
import pandas as pd
import numpy as np
import math
import timeit

from numpy import array

from preprocess_data import preprocess_credit, preprocess_income
from MW import fairDimReduction_MW
from utils import getObj

df = preprocess_credit("data/credit/creditdata.csv") # this method deals with all proprocessing and get one dataframe

dfCen=df-df.mean()

In [ ]:
#let's try 3 groups, with objective being the loss 
df1 = df[df['EDUCATION']<3.3]
df1 = df1-df1.mean()
df2 = df[(df['EDUCATION']>=3.3) & (df['EDUCATION']<=3.8)]
df2 = df2-df2.mean()
df3 = df[df['EDUCATION']>3.8]
df3 = df3-df3.mean()

#put data into k=3 matrices. Data matrix A --> A^T A
B1 = df1.values.T @ df1.values / len(df1)
B2 = df2.values.T @ df2.values / len(df2)
B3 = df3.values.T @ df3.values / len(df3)
B = [B1,B2,B3]

#example of how to call MW. This is on credit data with 3 groups
k=3
n=23
d=5
[X_last,X_avg,stats] = fairDimReduction_MW(n,k,d,B,eta=1, T=400, verbose = False,Obj='MM_Loss')
stats

In [ ]:
#the example aobve shows that MW actually converges pretty quickly. In fact, eta can be tune so the last iterate converges quickly for most datasets and value of n,d.
#But, not always... as an example below, 

#let's try 6 groups, with max min loss. This value of d we know MW last iterate cannot converge - we know from SDP solver
k=6
n=23
d=7

df1 = df[(df['EDUCATION']<3.3) & (df['SEX'] <= 2.5)]
df1 = df1-df1.mean()
df2 = df[(df['EDUCATION']>=3.3) & (df['EDUCATION']<=3.8) & (df['SEX'] <= 2.5)]
df2 = df2-df2.mean()
df3 = df[(df['EDUCATION']>3.8) & (df['SEX'] <= 2.5)]
df3 = df3-df3.mean()
df4 = df[(df['EDUCATION']<3.3) & (df['SEX'] > 2.5)]
df4 = df4-df4.mean()
df5 = df[(df['EDUCATION']>=3.3) & (df['EDUCATION']<=3.8) & (df['SEX'] > 2.5)]
df5 = df5-df5.mean()
df6 = df[(df['EDUCATION']>3.8) & (df['SEX'] > 2.5)]
df6 = df6-df6.mean()

B1 = df1.values.T @ df1.values / len(df1)
B2 = df2.values.T @ df2.values / len(df2)
B3 = df3.values.T @ df3.values / len(df3)
B4 = df4.values.T @ df4.values / len(df4)
B5 = df5.values.T @ df5.values / len(df5)
B6 = df6.values.T @ df6.values / len(df6)
B = [B1,B2,B3,B4,B5,B6]

#use MW to solve fair PCA
[X_last,X_avg,stats] = fairDimReduction_MW(n,k,d,B,eta=0.4, T=500, verbose = False,Obj='MM_Loss')
stats

In [ ]:
#another objective one can also try
n=23; d=7; k=6
[sol_X,sol_P,stats] = fairDimReduction_MW(n,k,d,B,eta=1, T=20, verbose = False,Obj='MM_Var')
stats

In [ ]:
#next, we can try the income data. First load the data
df = preprocess_income('data/income') #this preprocessing so it returns a list of 5 dataframes.
B = []
k=5 #There are 5 groups
for i in range(k): #data matrix format we need is A --> A^T A for method call later.
    B.append(df[i].values.T @ df[i].values / len(df[i]))
n=len(B[0]) #original dimension

In [ ]:
#example usage of MW, on income data
d=1
[_,__,stats] = fairDimReduction_MW(n,k,d,B,eta=0.01, T=500, verbose = False,Obj='MM_Loss')
stats

In [ ]:
#in some cases we do not expect MW last iterate to converge, since even SDP is not exact. e.g. for d=2 we know this is the case
d=2
[_,X_avg,stats] = fairDimReduction_MW(n,k,d,B,eta=0.05, T=1000, verbose = False,Obj='MM_Loss')
stats

In [ ]:
#in an example above, it turns out I can't make eta too big, else the alternation of MW solution is strong and the convergence is slow.
#so it seems that the income data requires about T=500 - 1000 iterations.